In [1]:
include("MySimplexQuad.jl")
using .MySimplexQuad

include("OrthoPols.jl")
using .OrthoPols

###############################################################

function rse(vec::Vector{T}) where T
    threshold = if T == BigFloat
        BigFloat("1e-60")
    else
        T(1e-14)
    end
    
    map(x -> abs(x) < threshold ? zero(T) : x, vec)
end

###############################################################


function Trans(point::Vector{T}) where T
    @assert length(point) == 2 "Input point must be a 2-element vector"

    # Define the vertices of the target triangle
    v1 = T[-1, -1 / sqrt(T(3))]   # (0,0) maps to this
    v2 = T[ 1, -1 / sqrt(T(3))]   # (1,0) maps to this
    v3 = T[ 0,  2 / sqrt(T(3))]   # (0,1) maps to this
    
    # Extract x and y from the input point
    x, y = point

    # Compute the affine transformation
    new_x = (v2[1] - v1[1]) * x + (v3[1] - v1[1]) * y + v1[1]
    new_y = (v2[2] - v1[2]) * x + (v3[2] - v1[2]) * y + v1[2]

    return T[new_x, new_y]
end

function TransBack(point::Vector{T}) where T
    @assert length(point) == 2 "Input point must be a 2-element vector"

    # Define the vertices of the target triangle
    v1 = T[-1, -1 / sqrt(T(3))]   # Maps to (0,0)
    v2 = T[ 1, -1 / sqrt(T(3))]   # Maps to (1,0)
    v3 = T[ 0,  2 / sqrt(T(3))]   # Maps to (0,1)
    
    # Extract x and y from the input point
    x, y = point

    # Compute the inverse affine transformation
    denominator = (v1[1] * (v2[2] - v3[2]) + v2[1] * (v3[2] - v1[2]) + v3[1] * (v1[2] - v2[2]))
    
    new_x = ((v2[2] - v3[2]) * (x - v1[1]) + (v3[1] - v2[1]) * (y - v1[2])) / denominator
    new_y = ((v3[2] - v1[2]) * (x - v2[1]) + (v1[1] - v3[1]) * (y - v2[2])) / denominator

    return T[new_x, new_y]
end

###############################################################

TransBack (generic function with 1 method)

In [13]:
using LinearAlgebra
using Optim
using Plots
using Optim
using LineSearches


freeparam = 7
order = 10
use_bigfloat = true

###############################################################

function rse(vec::Vector{T}) where T <: AbstractFloat
    threshold = if T == BigFloat
        BigFloat("1e-60")
    else
        T(1e-14)
    end
    
    map(x -> abs(x) < threshold ? zero(T) : x, vec)
end

###############################################################
p1 = use_bigfloat ? BigFloat[0.0, 0.0] : [0.0, 0.0]
p2 = use_bigfloat ? BigFloat[1.0, 0.0] : [1.0, 0.0]
p3 = use_bigfloat ? BigFloat[0.0, 1.0] : [0.0, 1.0]

m1 = (p2 .+ p3) ./ (use_bigfloat ? BigFloat(2.0) : 2.0)
m2 = (p1 .+ p3) ./ (use_bigfloat ? BigFloat(2.0) : 2.0)
m3 = (p1 .+ p2) ./ (use_bigfloat ? BigFloat(2.0) : 2.0)

b = (p1 .+ p2 .+ p3) ./ (use_bigfloat ? BigFloat(3.0) : 3.0)
###############################################################


# vertices
T1 = hcat(p1, p2, p3)
eval_T1(order) = rse(ortho2eva3(order, Trans(T1[:,1]))[1])+
                 rse(ortho2eva3(order, Trans(T1[:,2]))[1])+
                 rse(ortho2eva3(order, Trans(T1[:,3]))[1])

# edge midpoints
T2 = hcat(m1, m2, m3)
eval_T2(order) = rse(ortho2eva3(order, Trans(T2[:,1]))[1])+
                 rse(ortho2eva3(order, Trans(T2[:,2]))[1])+
                 rse(ortho2eva3(order, Trans(T2[:,3]))[1])

# midpoint
T3 = hcat(b)
eval_T3(order) = rse(ortho2eva3(order, Trans(T3[:,1]))[1])


# edge class
T4(a) = hcat(a * p1 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p1,
             a * p3 .+ (1 .- a) * p1,
             a * p1 .+ (1 .- a) * p3,
             a * p3 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p3)

dT4 = hcat(p1-p2,
           p2-p1,
           p3-p1,
           p1-p3,
           p3-p2,
           p2-p3)

eval_T4(order,a) = rse(ortho2eva3(order, Trans(T4(a)[:,1]))[1])+
                   rse(ortho2eva3(order, Trans(T4(a)[:,2]))[1])+
                   rse(ortho2eva3(order, Trans(T4(a)[:,3]))[1])+
                   rse(ortho2eva3(order, Trans(T4(a)[:,4]))[1])+
                   rse(ortho2eva3(order, Trans(T4(a)[:,5]))[1])+
                   rse(ortho2eva3(order, Trans(T4(a)[:,6]))[1])

# inner class, type 1
T5(a) = hcat(a * m1 .+ (1 .- a) * p1, 
             a * m2 .+ (1 .- a) * p2, 
             a * m3 .+ (1 .- a) * p3)

eval_T5(order,a) = rse(ortho2eva3(order, Trans(T5(a)[:,1]))[1])+
                   rse(ortho2eva3(order, Trans(T5(a)[:,2]))[1])+
                   rse(ortho2eva3(order, Trans(T5(a)[:,3]))[1])

# inner class, type 2
T6(a, b) =  hcat(b * (a * m1 .+ (1 - a) .* p1) .+ (1 - b) * (a * m2 .+ (1 - a) .* p2),
                 b * (a * m1 .+ (1 - a) .* p1) .+ (1 - b) * (a * m3 .+ (1 - a) .* p3),
                 b * (a * m3 .+ (1 - a) .* p3) .+ (1 - b) * (a * m2 .+ (1 - a) .* p2),
                 b * (a * m2 .+ (1 - a) .* p2) .+ (1 - b) * (a * m1 .+ (1 - a) .* p1),
                 b * (a * m3 .+ (1 - a) .* p3) .+ (1 - b) * (a * m1 .+ (1 - a) .* p1),
                 b * (a * m2 .+ (1 - a) .* p2) .+ (1 - b) * (a * m3 .+ (1 - a) .* p3))

eval_T6(order,a,b) = rse(ortho2eva3(order, Trans(T6(a, b)[:,1]))[1])+
                     rse(ortho2eva3(order, Trans(T6(a, b)[:,2]))[1])+
                     rse(ortho2eva3(order, Trans(T6(a, b)[:,3]))[1])+
                     rse(ortho2eva3(order, Trans(T6(a, b)[:,4]))[1])+
                     rse(ortho2eva3(order, Trans(T6(a, b)[:,5]))[1])+
                     rse(ortho2eva3(order, Trans(T6(a, b)[:,6]))[1])

# System setup
function A(a)
    hcat(
        eval_T1(order), # trig vertices
        # eval_T2(order), # edge midpoints

        eval_T4(order,a[1]), # edge class
        eval_T4(order,a[2]), # edge class
        
        eval_T5(order,a[3]),
        eval_T5(order,a[4]),
        eval_T5(order,a[5]),
        
        eval_T3(order), # trig midpoint

        eval_T6(order,a[6],a[7]),
        # eval_T5(order,a[8]),
        )'
end

# function dA(a,order)

function rhs() # <: AbstractFloat
    T = use_bigfloat ? BigFloat : Float64
    X, W = mysimplexquad(T, order, 2)
    w = zeros(T, div((order + 1) * (order + 2), 2))  # Initialize w with the correct type and size
    
    for k = 1:length(W)
        pols = ortho2eva3(order, Trans(T.(X[k,:])))[1]
        w .+= 2*W[k] .* pols
    end
    return w
end

f(a) = norm(A(a)'*((A(a) * A(a)')\(A(a) * rhs(BigFloat)))-rhs(BigFloat))
g(a) = A(a)'*((A(a) * A(a)')\(A(a) * rhs(BigFloat)))-rhs(BigFloat)


g (generic function with 1 method)

In [14]:
# using Zygote
# z(x) = 5x + 3


# f(a) = norm(A(a)'*((A(a) * A(a)')\(A(a) * rhs(BigFloat,order)))-rhs(BigFloat,order))
A([0.1, 0.2, 0.1, 0.34, 0.33, 0.32, 0.31])
rhs()
# f'([0.1, 0.2, 0.35, 0.34, 0.33, 0.32, 0.31])
# f'(2.0)

# plot(sort(abs.(g([0.1, 0.2, 0.1, 0.34, 0.33, 0.32, 0.31]))))

66-element Vector{BigFloat}:
  0.7598356856515925473311877506545453353967734488873103186112886537003222331710314
  4.159529620481035243398973377669098699682042363027318338958682179186669411445028e-77
 -6.713271650249197189265171955848748104109703408292265607889519494388866284489543e-78
 -6.807729743820542677355335179348127474418784662278287435538216251093835257336632e-77
  7.556647485707639047213057879950349624726500318881746211895740536397517827767124e-78
 -4.368686827674728824170049086846295876795007996853509528752224997604814994177868e-77
  1.632775617447543436987107149060700543914118818901234449356043937328749387785396e-77
 -3.535431502241788268517537793833913574425612649191102691994078608100267269419619e-77
 -1.012051002550130229537463108921921824740156292707376724807465250410381851933097e-78
  4.554229511475586032918583990148648211330703317183195261633593626846718333698936e-78
  ⋮
 -1.086268076070473113036877070242862758554434420839251017960012702107143187741524e-77
 -5.400978850

In [ ]:
# Define the bounds
# lower_bounds = BigFloat[0.0, 0.0, 0.0]
# upper_bounds = BigFloat[Inf, Inf, Inf]

# Set up the optimization problem with bounds
# result = optimize(f, lower_bounds, upper_bounds, BigFloat[0.1, 0.1, 0.2], Fminbox(NelderMead()), Optim.Options(g_tol = 1e-63, iterations = 100_000, show_trace = false))

# result = optimize(f, 
#             lower_bounds, 
#             upper_bounds, 
#             BigFloat[0.1, 0.2, 0.3],
#             Fminbox(NelderMead()), 
#             Optim.Options(g_tol = 1e-63, iterations = 100_000, show_trace = true, show_every = 100))

In [ ]:
# a = Optim.minimizer(result)

# a = BigFloat[0.1, 0.2, 0.3]

min_val = 0.2
max_val = 0.7
a = min_val .+ (max_val - min_val) .* BigFloat.(rand(freeparam))

# display(a)
# display(g(a))
using Plots
plot()
display(A(a,order))

In [ ]:
# println(a)

# td = TwiceDifferentiable(f, a; autodiff = :forward);

# # result = optimize(td, a, Newton(; linesearch = LineSearches.MoreThuente()), Optim.Options(g_tol = 1e-100, iterations = 1000, show_trace = true, show_every = 1, time_limit = 1000))
# # result = optimize(td, a, Newton(; linesearch = LineSearches.BackTracking(), alphaguess = LineSearches.InitialStatic()), Optim.Options(g_tol = 1e-15, iterations = 1000, show_trace = true, show_every = 1, time_limit = 1000))

# result = optimize(td, a, Newton(; linesearch = LineSearches.BackTracking(), alphaguess = LineSearches.InitialStatic()), Optim.Options(g_tol = 1e-63, iterations = 50, show_trace = true, show_every = 10, time_limit = 1000))

# println(result)

# a = Optim.minimizer(result)
# using Base.Threads

# using Distributed

# num_runs = 5

# @threads  for i in 1:num_runs

# a = BigFloat[0.3, 0.2, 0.1]

of = TwiceDifferentiable(f, a; autodiff = :forward)
# method = Newton(; linesearch = LineSearches.MoreThuente())
method = Newton(; linesearch = LineSearches.BackTracking(order=3))


# of = OnceDifferentiable(f, a; autodiff = :forward);
# method = GradientDescent()

options = Optim.Options(g_tol = 1e-50, iterations = 20, show_trace = true, show_every = 1, time_limit = 1000)


# lower_bounds = fill(BigFloat("0"), freeparam)
# # upper_bounds = fill(Inf, freeparam)

result = optimize(of, a, method, options)
# # result = optimize(of,lower_bounds, upper_bounds, a, method, options)
a = Optim.minimizer(result)

print(a)

In [ ]:
a = Optim.minimizer(result)
display(f(a))
display(a)

In [ ]:
using Plots
# gr()  # Set the backend to GR
plot()
# Define the vertices of the triangle
x = [p1[1], p2[1], p3[1], p1[1]]
y = [p1[2], p2[2], p3[2], p1[2]]



# eval_T1(order), # trig vertices
# # eval_T2(order), # edge midpoints

# eval_T4(order,a[1]), # edge class
# eval_T4(order,a[2]), # edge class

# eval_T5(order,a[3]),
# eval_T5(order,a[4]),
# eval_T5(order,a[5]),

# eval_T3(order), # trig midpoint

# eval_T6(order,a[6],a[7]),
# # eval_T5(order,a[8]),




# Plot the triangle
plot!(x, y, seriestype = :shape, fillalpha = 0.3, linecolor = :blue, legend = false, aspect_ratio = :equal)

scatter!(T1[1, :], T1[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)
# scatter!(T2[1, :], T2[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)
scatter!(T3[1, :], T3[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)

scatter!(T4(a[1])[1, :], T4(a[1])[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)
scatter!(T4(a[2])[1, :], T4(a[2])[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)

scatter!(T5(a[3])[1, :], T5(a[3])[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)
scatter!(T5(a[4])[1, :], T5(a[4])[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)
scatter!(T5(a[5])[1, :], T5(a[5])[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)

scatter!(T6(a[6],a[7])[1, :], T6(a[6],a[7])[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)

# scatter!(T5(a[8])[1, :], T5(a[8])[2, :], markersize = 3, markercolor = :blue, legend = false, aspect_ratio = :equal)

# Add plot title and labels
xlabel!("x")
ylabel!("y")

# savefig("kek.png")

In [ ]:
vertices = [
    0.0 0.0
    1.0 0.0
    0.0 1.0
]

# Need at least 4 vertices for plotting?
vertices1 = [vertices; vertices[1:1, :]]

using Plots
plot()

# for npoints in [1,2]
npoints = 4
    X, W = mysimplexquad(BigFloat, npoints, 2)

    # Plot the triangle
    plot!(vertices1'[1,:], vertices1'[2,:], seriestype = :shape, fillalpha = 0.3, linecolor = :blue, legend = false, aspect_ratio = :equal)
    scatter!(X[:, 1], X[:, 2], markersize = 8, markercolor = :red, legend = false, aspect_ratio = :equal)

    # Add plot title and labels
    xlabel!("x")
    ylabel!("y")
# end

In [ ]:
w = A(a,order)'*((A(a,order) * A(a,order)')\(A(a,order) * rhs(BigFloat,order)))
rhs(BigFloat,order)
# sum(w)
w
# K = A(a,order)'
# sum(K,dims=2)
# sum(,1)